In [1]:
# Necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(font_scale=1.5, style = 'whitegrid', color_codes=True)

import nltk
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax
import torch

import ast
from datetime import datetime
from tqdm import tqdm
import time

# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation



pd.set_option('max_colwidth', 200)
pd.set_option('display.max_rows', 100)

In [2]:
df = pd.read_csv('chatgpt1.csv')

In [3]:
# Prints any 5 random
df.sample(5)

,Datetime,Tweet Id,Text,Username,Permalink,User,Outlinks,CountLinks,ReplyCount,RetweetCount,LikeCount,QuoteCount,ConversationId,Language,Source,Media,QuotedTweet,MentionedUsers,hashtag,hastag_counts
13884,2023-01-23 04:25:42+00:00,1617378013863358464,"ChatGPT also passed a medical licensing exam(USMLE). However, one doesn’t really need AI to pass an exam if they were given access to internet. Searching Google would still get you to pass the exa...",DarkrumD,https://twitter.com/DarkrumD/status/1617378013863358464,https://twitter.com/DarkrumD,['https://www.ft.com/content/7229ba86-142a-49f6-9821-f55c07536b7c'],['https://t.co/ZWsYZHrvtl'],0,0,1,0,1617378013863358464,en,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",NaN,NaN,NaN,[],0
1658,2023-01-22 15:12:22+00:00,1617178363705057280,"When it came to questions on fundamental operations management and process analysis, Chat GPT ""did a great job in education.""\n\n#ChatGPT #Technology #Education #TechNews\n\nChatGPT Passes MBA Exa...",theinsightnow,https://twitter.com/theinsightnow/status/1617178363705057280,https://twitter.com/theinsightnow,['https://theinsighttoday.com/chatgpt-passes-mba-exam-claims-professor/'],['https://t.co/7WBrHlAGgT'],0,0,0,0,1617178363705057280,en,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",NaN,NaN,NaN,"['#ChatGPT', '#Technology', '#Education', '#TechNews']",4
17952,2023-01-23 08:41:45+00:00,1617442450423042051,"@ChatGPTBot @SDelzon This is a very, very long story that I'm not sure even you, @chatGPT, want to know 😅",torresruizjm,https://twitter.com/torresruizjm/status/1617442450423042051,https://twitter.com/torresruizjm,NaN,NaN,0,0,0,0,1617271035962744833,en,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",NaN,NaN,"[User(username='ChatGPTBot', id=1598922281434103808, displayname='ChatGPT ✨', description=None, rawDescription=None, descriptionUrls=None, verified=None, created=None, followersCount=None, friends...",[],0
32254,2023-01-23 17:37:37+00:00,1617577304892243985,"De impact van ChatGPT op taalonderwijs. Eigenlijk heel analoog aan het verschijnen van e.g. rekenmachine, laptops, smartphones en hun impact op gerelateerd onderwijs. \nInzetten op inzicht, compet...",RobinLombaert,https://twitter.com/RobinLombaert/status/1617577304892243985,https://twitter.com/RobinLombaert,['https://excel.thomasmore.be/2023/01/artificiele-intelligentie-wat-moet-je-als-leerkracht-weten-over-chatgpt/'],['https://t.co/FFQLDwOACs'],0,0,0,0,1617577304892243985,nl,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",NaN,NaN,NaN,[],0
27532,2023-01-23 15:22:44+00:00,1617543362051198978,مایکروسافت «چند میلیارد دلار» در شرکت سازنده ChatGPT سرمایه‌گذاری می‌کند \n \n ۱. قبلاً گزارشی منتشر شده بود که از احتمال سرمایه‌گذاری 10 میلیارد دلاری مایکروسافت در OpenAI حکایت داشت. حالا مایک...,Digiato,https://twitter.com/Digiato/status/1617543362051198978,https://twitter.com/Digiato,NaN,NaN,2,2,83,0,1617543362051198978,fa,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>","[Photo(previewUrl='https://pbs.twimg.com/media/FnKrHWrWIAAkPuJ?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/FnKrHWrWIAAkPuJ?format=jpg&name=large')]",NaN,NaN,[],0


In [4]:
# Prints 10 in order
df.head(10)

,Datetime,Tweet Id,Text,Username,Permalink,User,Outlinks,CountLinks,ReplyCount,RetweetCount,LikeCount,QuoteCount,ConversationId,Language,Source,Media,QuotedTweet,MentionedUsers,hashtag,hastag_counts
0,2023-01-22 13:44:34+00:00,1617156270871699456,ChatGPTで遊ぶの忘れてた！！\n書類作るコード書いてみてほしいのと、\nどこまで思考整理付き合ってくれるのかと、\nウミガメのスープやってみたい。\nあとは調べ物してくれ。\n事務所でちょいといじったが、とりあえず色々やってみたい。,mochico0123,https://twitter.com/mochico0123/status/1617156270871699456,https://twitter.com/mochico0123,NaN,NaN,1,0,5,0,1617156270871699456,ja,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",NaN,NaN,NaN,[],0
1,2023-01-22 13:44:39+00:00,1617156291046133761,@AlexandrovnaIng Prohibition of ChatGPT has been added to the honor code of my daughter’s school,Caput_LupinumSG,https://twitter.com/Caput_LupinumSG/status/1617156291046133761,https://twitter.com/Caput_LupinumSG,NaN,NaN,1,0,5,0,1617148639993806848,en,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",NaN,NaN,"[User(username='AlexandrovnaIng', id=2827059006, displayname='Alexandrovna', description=None, rawDescription=None, descriptionUrls=None, verified=None, created=None, followersCount=None, friendsC...",[],0
2,2023-01-22 13:44:44+00:00,1617156308926349312,"Schaut Euch an, was @fobizz @DianaKnodel alles ⬇️ auf die Piste bringt, um Euch mitzunehmen. \n\nMorgen startet @reg_schulz - und am Di um 12 besprechen wir #ChatGPT als kritische FreundInnen im L...",ciffi,https://twitter.com/ciffi/status/1617156308926349312,https://twitter.com/ciffi,"['https://us02web.zoom.us/webinar/register/8016723032829/WN_w_ObSIzWSL2X2EEOThSbdw', 'https://twitter.com/DianaKnodel/status/1617153572654944259']","['https://t.co/DsoeVJrPBp', 'https://t.co/HflTruResk']",0,0,4,0,1617156308926349312,de,"<a href=""http://twitter.com/#!/download/ipad"" rel=""nofollow"">Twitter for iPad</a>","[Photo(previewUrl='https://pbs.twimg.com/media/FnFLF3SWIAAtThd?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/FnFLF3SWIAAtThd?format=jpg&name=large')]",https://twitter.com/DianaKnodel/status/1617153572654944259,"[User(username='fobizz', id=884708145792253952, displayname='fobizz', description=None, rawDescription=None, descriptionUrls=None, verified=None, created=None, followersCount=None, friendsCount=No...",['#ChatGPT'],1
3,2023-01-22 13:44:49+00:00,1617156332297256961,Bow down to chatGPT 🫡..... https://t.co/ENTSzi2AQ9,Vishwasrisiri,https://twitter.com/Vishwasrisiri/status/1617156332297256961,https://twitter.com/Vishwasrisiri,['https://twitter.com/agadmator/status/1617155017638543361'],['https://t.co/ENTSzi2AQ9'],0,0,2,0,1617156332297256961,en,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",NaN,https://twitter.com/agadmator/status/1617155017638543361,NaN,[],0
4,2023-01-22 13:44:52+00:00,1617156345064570880,"Profilinde vatan, Türkiye falan yazan bireylerin ilk koruduğu şey ülkenin dini falan. Asla şaşmaz bu. Chatgpt yaz bunu bi kenara 😂",0xGenetikciniz,https://twitter.com/0xGenetikciniz/status/1617156345064570880,https://twitter.com/0xGenetikciniz,NaN,NaN,0,0,4,0,1617156345064570880,tr,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",NaN,NaN,NaN,[],0
5,2023-01-22 13:45:00+00:00,1617156376983207937,ChatGPT’nin bilinmeyen arka planı: Ucuz iş gücü\n\nÇeviri: Evrim Şaşmaz\n\nhttps://t.co/daAyOpRLiM,GazeteKarinca,https://twitter.com/GazeteKarinca/status/1617156376983207937,https://twitter.com/GazeteKarinca,['https://gazetekarinca.com/chatgptnin-bilinmeyen-arka-plani-ucuz-is-gucu/'],['https://t.co/daAyOpRLiM'],0,0,1,0,1617156376983207937,tr,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,NaN,[],0
6,2023-01-22 13:45:03+00:00,1617156389217894400,ChatGPT runs 10K Nvidia training GPUs with potential for thousands more https://t.co/Uhq62t0Uw4,FierceElectron,https://twitter.com/FierceElectron/status/1617156389217894400,https://twitter.com/FierceElectr

In [5]:
df.isna().sum()

Datetime              0
Tweet Id              0
Text                  0
Username              0
Permalink             0
User                  0
Outlinks          30059
CountLinks        30059
ReplyCount            0
RetweetCount          0
LikeCount             0
QuoteCount            0
ConversationId        0
Language              0
Source                0
Media             40499
QuotedTweet       46438
MentionedUsers    32832
hashtag               0
hastag_counts         0
dtype: int64

In [11]:
df.isna().sum()/len(df)*100

Datetime           0.000000
Tweet Id           0.000000
Text               0.000000
Username           0.000000
Permalink          0.000000
User               0.000000
Outlinks          60.116798
CountLinks        60.116798
ReplyCount         0.000000
RetweetCount       0.000000
LikeCount          0.000000
QuoteCount         0.000000
ConversationId     0.000000
Language           0.000000
Source             0.000000
Media             80.996380
QuotedTweet       92.874143
MentionedUsers    65.662687
hashtag            0.000000
hastag_counts      0.000000
dtype: float64

In [12]:
df.dtypes

Datetime          object
Tweet Id           int64
Text              object
Username          object
Permalink         object
User              object
Outlinks          object
CountLinks        object
ReplyCount         int64
RetweetCount       int64
LikeCount          int64
QuoteCount         int64
ConversationId     int64
Language          object
Source            object
Media             object
QuotedTweet       object
MentionedUsers    object
hashtag           object
hastag_counts      int64
dtype: object

In [13]:
# Display the first few rows of the DataFrame
df.head()

,Datetime,Tweet Id,Text,Username,Permalink,User,Outlinks,CountLinks,ReplyCount,RetweetCount,LikeCount,QuoteCount,ConversationId,Language,Source,Media,QuotedTweet,MentionedUsers,hashtag,hastag_counts
0,2023-01-22 13:44:34+00:00,1617156270871699456,ChatGPTで遊ぶの忘れてた！！\n書類作るコード書いてみてほしいのと、\nどこまで思考整理付き合ってくれるのかと、\nウミガメのスープやってみたい。\nあとは調べ物してくれ。\n事務所でちょいといじったが、とりあえず色々やってみたい。,mochico0123,https://twitter.com/mochico0123/status/1617156270871699456,https://twitter.com/mochico0123,NaN,NaN,1,0,5,0,1617156270871699456,ja,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",NaN,NaN,NaN,[],0
1,2023-01-22 13:44:39+00:00,1617156291046133761,@AlexandrovnaIng Prohibition of ChatGPT has been added to the honor code of my daughter’s school,Caput_LupinumSG,https://twitter.com/Caput_LupinumSG/status/1617156291046133761,https://twitter.com/Caput_LupinumSG,NaN,NaN,1,0,5,0,1617148639993806848,en,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",NaN,NaN,"[User(username='AlexandrovnaIng', id=2827059006, displayname='Alexandrovna', description=None, rawDescription=None, descriptionUrls=None, verified=None, created=None, followersCount=None, friendsC...",[],0
2,2023-01-22 13:44:44+00:00,1617156308926349312,"Schaut Euch an, was @fobizz @DianaKnodel alles ⬇️ auf die Piste bringt, um Euch mitzunehmen. \n\nMorgen startet @reg_schulz - und am Di um 12 besprechen wir #ChatGPT als kritische FreundInnen im L...",ciffi,https://twitter.com/ciffi/status/1617156308926349312,https://twitter.com/ciffi,"['https://us02web.zoom.us/webinar/register/8016723032829/WN_w_ObSIzWSL2X2EEOThSbdw', 'https://twitter.com/DianaKnodel/status/1617153572654944259']","['https://t.co/DsoeVJrPBp', 'https://t.co/HflTruResk']",0,0,4,0,1617156308926349312,de,"<a href=""http://twitter.com/#!/download/ipad"" rel=""nofollow"">Twitter for iPad</a>","[Photo(previewUrl='https://pbs.twimg.com/media/FnFLF3SWIAAtThd?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/FnFLF3SWIAAtThd?format=jpg&name=large')]",https://twitter.com/DianaKnodel/status/1617153572654944259,"[User(username='fobizz', id=884708145792253952, displayname='fobizz', description=None, rawDescription=None, descriptionUrls=None, verified=None, created=None, followersCount=None, friendsCount=No...",['#ChatGPT'],1
3,2023-01-22 13:44:49+00:00,1617156332297256961,Bow down to chatGPT 🫡..... https://t.co/ENTSzi2AQ9,Vishwasrisiri,https://twitter.com/Vishwasrisiri/status/1617156332297256961,https://twitter.com/Vishwasrisiri,['https://twitter.com/agadmator/status/1617155017638543361'],['https://t.co/ENTSzi2AQ9'],0,0,2,0,1617156332297256961,en,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",NaN,https://twitter.com/agadmator/status/1617155017638543361,NaN,[],0
4,2023-01-22 13:44:52+00:00,1617156345064570880,"Profilinde vatan, Türkiye falan yazan bireylerin ilk koruduğu şey ülkenin dini falan. Asla şaşmaz bu. Chatgpt yaz bunu bi kenara 😂",0xGenetikciniz,https://twitter.com/0xGenetikciniz/status/1617156345064570880,https://twitter.com/0xGenetikciniz,NaN,NaN,0,0,4,0,1617156345064570880,tr,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",NaN,NaN,NaN,[],0


In [14]:
# Get information about the DataFrame (column names, data types, non-null values, etc.)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50001 entries, 0 to 50000
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Datetime        50001 non-null  object
 1   Tweet Id        50001 non-null  int64 
 2   Text            50001 non-null  object
 3   Username        50001 non-null  object
 4   Permalink       50001 non-null  object
 5   User            50001 non-null  object
 6   Outlinks        19942 non-null  object
 7   CountLinks      19942 non-null  object
 8   ReplyCount      50001 non-null  int64 
 9   RetweetCount    50001 non-null  int64 
 10  LikeCount       50001 non-null  int64 
 11  QuoteCount      50001 non-null  int64 
 12  ConversationId  50001 non-null  int64 
 13  Language        50001 non-null  object
 14  Source          50001 non-null  object
 15  Media           9502 non-null   object
 16  QuotedTweet     3563 non-null   object
 17  MentionedUsers  17169 non-null  object
 18  hashta

In [15]:
df.describe()

,Tweet Id,ReplyCount,RetweetCount,LikeCount,QuoteCount,ConversationId,hastag_counts
count,5.000100e+04,50001.000000,50001.000000,50001.000000,50001.000000,5.000100e+04,50001.000000
mean,1.617493e+18,0.929141,1.498510,9.696326,0.219536,1.617205e+18,0.783304
std,1.725682e+14,23.251710,46.030058,313.524215,10.356329,1.005075e+16,1.975040
min,1.617156e+18,0.000000,0.000000,0.000000,0.000000,6.493609e+17,0.000000
25%,1.617354e+18,0.000000,0.000000,0.000000,0.000000,1.617302e+18,0.000000
50%,1.617525e+18,0.000000,0.000000,0.000000,0.000000,1.617504e+18,0.000000
75%,1.617625e+18,1.000000,0.000000,2.000000,0.000000,1.617607e+18,1.000000
max,1.617779e+18,3098.000000,6815.000000,56073.000000,1947.000000,1.617779e+18,28.000000


In [16]:
df.isnull().sum()

Datetime              0
Tweet Id              0
Text                  0
Username              0
Permalink             0
User                  0
Outlinks          30059
CountLinks        30059
ReplyCount            0
RetweetCount          0
LikeCount             0
QuoteCount            0
ConversationId        0
Language              0
Source                0
Media             40499
QuotedTweet       46438
MentionedUsers    32832
hashtag               0
hastag_counts         0
dtype: int64

In [17]:
# Removing duplicates
# Check for duplicates
df.duplicated().sum()

0

In [18]:
df.isna().sum()

Datetime              0
Tweet Id              0
Text                  0
Username              0
Permalink             0
User                  0
Outlinks          30059
CountLinks        30059
ReplyCount            0
RetweetCount          0
LikeCount             0
QuoteCount            0
ConversationId        0
Language              0
Source                0
Media             40499
QuotedTweet       46438
MentionedUsers    32832
hashtag               0
hastag_counts         0
dtype: int64

In [19]:
print(df.columns)

Index(['Datetime', 'Tweet Id', 'Text', 'Username', 'Permalink', 'User',
       'Outlinks', 'CountLinks', 'ReplyCount', 'RetweetCount', 'LikeCount',
       'QuoteCount', 'ConversationId', 'Language', 'Source', 'Media',
       'QuotedTweet', 'MentionedUsers', 'hashtag', 'hastag_counts'],
      dtype='object')


In [20]:
df = df[['Text', 'Language']]
df = df.dropna()

In [21]:
x = df['Text']
y = df['Language']

In [22]:
y.unique()

array(['ja', 'en', 'de', 'tr', 'pl', 'fr', 'es', 'pt', 'lo', 'no', 'ca',
       'zh', 'qme', 'th', 'ne', 'ko', 'und', 'nl', 'fa', 'it', 'da', 'fi',
       'eu', 'hi', 'ar', 'sv', 'in', 'ru', 'qht', 'tl', 'hu', 'cs', 'uk',
       'iw', 'et', 'cy', 'bg', 'ht', 'el', 'vi', 'sl', 'kn', 'ro', 'lt',
       'ur', 'zxx', 'ml', 'mr', 'lv', 'gu', 'qam', 'is', 'ta', 'te', 'pa',
       'sd', 'am', 'sr', 'hy', 'or', 'bn'], dtype=object)

In [26]:
# Splitting the dataset

from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Display the shape of the training and testing sets
print("Training set shape:", train_df.shape)
print("Testing set shape:", test_df.shape)

Training set shape: (40000,)
Testing set shape: (10001,)


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [29]:
# Continue with the TF-IDF vectorization
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [33]:
X_train_tfidf.shape

(40000, 113153)

In [34]:
# Decision Tree

# # Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train_tfidf,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test_tfidf)

In [35]:
y_pred

array(['en', 'ja', 'en', ..., 'qme', 'en', 'en'], dtype=object)

In [36]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8679132086791321


In [37]:
# Data Processing
import pandas as pd
import numpy as np

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image


In [38]:
rf = RandomForestClassifier()
rf.fit(X_train_tfidf, y_train)

RandomForestClassifier()

In [39]:
y_pred = rf.predict(X_train_tfidf)

In [40]:
y_pred

array(['en', 'en', 'en', ..., 'en', 'es', 'en'], dtype=object)

In [43]:
from sklearn.metrics import accuracy_score

# Ensure y_pred has the same number of samples as y_test
y_pred = clf.predict(X_test_tfidf)  # Replace 'clf' with your trained model object

# Convert y_pred from a multi-dimensional array to a 1D array if necessary
y_pred = y_pred.flatten()

# Trim y_test to match the size of y_pred
y_test_trimmed = y_test[:len(y_pred)]

# Calculate the accuracy score
accuracy = accuracy_score(y_test_trimmed, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8679132086791321


In [44]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8679132086791321


In [45]:
# KNN 
import sklearn
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=50)
knn.fit(X_train_tfidf, y_train)

KNeighborsClassifier(n_neighbors=50)

In [46]:
y_pred = knn.predict(X_train_tfidf)

/Users/student/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [47]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

ValueError: Found input variables with inconsistent numbers of samples: [10001, 40000]